We're using simplfied attention mechanism and we're using a 3 dimensional vector for all these.

# **Attention Mechanism**

## **Simplified Attention**

In [ ]:
import torch

In [ ]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # Journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.1],  # one
        [0.05, 0.80, 0.55]  # step
    ]
)

**DAMNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN**

It was the dot product

In [ ]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)

attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

Normalizing

Not a greate way tbh. Let's do Softmax. Why ? IN NOTES !!!!

Let's skip naive softmax and just do pytorch softmax

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [ ]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, attn_weight in enumerate(attn_weights_2):
  context_vec_2 += attn_weight * inputs[i]

In [ ]:
context_vec_2

tensor([0.4419, 0.6515, 0.5683])

In [ ]:
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i][j] = torch.dot(x_i, x_j)

attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

But the above is VERY EXPENSIVE computationally.

And hnce we use MUCH MORE EFFICIENT thing

In [ ]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [ ]:
context_vec = attn_weights @ inputs
context_vec

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Moving to self-attention with trainable weights

In [ ]:
import torch

In [ ]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # Journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.1],  # one
        [0.05, 0.80, 0.55]  # step
    ]
)

We have query, key and weight matrices in random values

- Parameter is basically to tell PyTorch that this tensor is "trainable" tensor. More on that, ask GPT.

Lets do queries, keys and values matrices for x_2 (Journey).

We get the query of all the inputs

Lets get the attention scores matrix of all inputs with every other inputs.
- we just multiply queries (6x2) with keys (2x6) to get the nice trainable matrix of attention scores that is (6x6)

Lets start by getting the attention weights.

We divide the atention scores with sqrt(d_of_keys) and then apply softmax to get the attention weights.

Lets implement the Self Attention classes.

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__()
    self.w_query = nn.Parameter(torch.rand(d_in, d_out))
    self.w_key = nn.Parameter(torch.rand(d_in, d_out))
    self.w_value = nn.Parameter(torch.rand(d_in, d_out))

  def forward(self, x):
    queries = x @ self.w_query
    keys = x @ self.w_key
    values = x @ self.w_value

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

    context_vec = attn_weights @ values

    return context_vec

In [ ]:
att_1 = SelfAttention_v1(inputs.shape[-1], 2)
att_1(inputs)

tensor([[0.7966, 0.5731],
        [0.8082, 0.5837],
        [0.8076, 0.5832],
        [0.7848, 0.5610],
        [0.7835, 0.5592],
        [0.7921, 0.5685]], grad_fn=<MmBackward0>)

We can use nn.Linear, instead of nn.Parameter() as it automatically initializes the weight matrices (internally call nn.Parameter()) which is good for computations and also, it is much stable and effective at model training.

In [ ]:
class SelfAttention_v2(nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__()
    self.w_query = nn.Linear(d_in, d_out, bias=False)
    self.w_key = nn.Linear(d_in, d_out, bias=False)
    self.w_value = nn.Linear(d_in, d_out, bias=False)

  def forward(self, x):
    keys = self.w_key(x)
    queries = self.w_query(x)
    values = self.w_value(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

    context_vec = attn_weights @ values

    return context_vec

In [ ]:
att_2 = SelfAttention_v2(inputs.shape[-1], 2)
att_2(inputs)

tensor([[ 0.2334, -0.1073],
        [ 0.2312, -0.1064],
        [ 0.2314, -0.1064],
        [ 0.2305, -0.1062],
        [ 0.2339, -0.1065],
        [ 0.2291, -0.1061]], grad_fn=<MmBackward0>)

## **Causal Attention**

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # Journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.1],  # one
        [0.05, 0.80, 0.55]  # step
    ]
)

We get triangular matrix using torch.tril and then NEVERMIND

WE PASS IN THE ATTENTION WEIGHTS TO TORCH.TRIL TO GET MASK. HAHAHA, HE MISSED IT.

Normalizing

We make 1s and 0s in mask tensor with upper side of diagonal with 1s and lower with 0s.
We than convert the att_scores part to -inf wherever there is 1 in mask.

And now softmax

Lets create a class now

Lets do batches

In [3]:
class CasualAttention(nn.Module):

  # We define output dimension, context_length (length of one item in the batch), dropout rate
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
    super().__init__()
    self.d_out = d_out

    # key, query and value weight matrices as well as dropout layer
    self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)

    # We initialize a buffer which is basically a upper traingular matrix
    self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

  def forward(self, x):

    # getting batch, number of tokens or context length as well as the embedding vector size
    batch, num_token, d_in = x.shape # Batches b

    # keys, values and queries matrices
    keys = self.w_key(x)
    values = self.w_value(x)
    queries = self.w_query(x)

    # Getting attention scores by transposing keys and using last two dimensions
    attn_scores = queries @ keys.transpose(1,2)

    # Changing 1s in the attention scores matrix to -inf
    # we also use [:num_token, :num_token], because what if we ONLY need 4x4 as the number of tokens are less than the context_size
    attn_scores = attn_scores.masked_fill(self.mask.bool()[:num_token, :num_token], -torch.inf)

    # Applying softmax to zero out the -infs and basically hiding the future tokens by creating attn_scores of those = 0
    attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
    attn_weights = self.dropout(attn_weights) # Applying dropout

    context_vector = attn_weights @ values  # Context vectors

    return context_vector

In [4]:
batch = torch.stack((inputs, inputs))

In [5]:
cas_attn = CasualAttention(
  d_in=3,
  d_out=2,
  context_length=6,
  dropout=0
)

In [6]:
cas_attn(batch)

tensor([[[ 0.3541, -0.2838],
         [ 0.1350, -0.2300],
         [ 0.0593, -0.2145],
         [ 0.0101, -0.1688],
         [ 0.0072, -0.1969],
         [-0.0232, -0.1566]],

        [[ 0.3541, -0.2838],
         [ 0.1350, -0.2300],
         [ 0.0593, -0.2145],
         [ 0.0101, -0.1688],
         [ 0.0072, -0.1969],
         [-0.0232, -0.1566]]], grad_fn=<UnsafeViewBackward0>)

## **MULTIHEAD ATTENTION MECHANISM**



we create multiple instances of casual self attention mechanism and concat them.

In [22]:
class MultiHeadAttentionWrapper(nn.Module):

  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
    super().__init__()

    # we create a list of classes and store it in heads using nn.Module.
    # there are basically num_heads number of heads or CasualAttention classes initialized
    self.heads = nn.ModuleList(
        [CasualAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
    )

  # We concat all the context vectors we get after passing x to each head in heads
  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim=-1)

Well the above thing is very inefficient as we are initializing one vector and that getting context for 1.
Then other. then other. then other.

But we will make it better on next lecture cuz that shit is official as fck. That is what openai used for gpts.

In [23]:
ma = MultiHeadAttentionWrapper(3, 2, 6, 0000, 2)

In [24]:
ma(batch)

tensor([[[-0.3051,  0.5048,  0.0693,  0.4790],
         [-0.4397,  0.5333,  0.0808,  0.3125],
         [-0.4840,  0.5410,  0.0903,  0.2603],
         [-0.4409,  0.4772,  0.0673,  0.1920],
         [-0.4328,  0.4419,  0.1307,  0.2116],
         [-0.4174,  0.4301,  0.0840,  0.1605]],

        [[-0.3051,  0.5048,  0.0693,  0.4790],
         [-0.4397,  0.5333,  0.0808,  0.3125],
         [-0.4840,  0.5410,  0.0903,  0.2603],
         [-0.4409,  0.4772,  0.0673,  0.1920],
         [-0.4328,  0.4419,  0.1307,  0.2116],
         [-0.4174,  0.4301,  0.0840,  0.1605]]], grad_fn=<CatBackward0>)

## Better MultiHeadAttention

In [7]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # Basically a thing between if-else and try-catch block
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        # we keep a output dimension, number of heads and dimension of single head's context_vec
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        # Weights for q, k, v
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Dropout as well as a buffer for masked triangular matrix
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

  def forward(self, x):
    b, num_token, d_in = x.shape

    # getting k, q, v
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    # (b, num_token, d_out) -> (b, num_token, num_head, head_dim) for queries, keys, values
    keys = keys.view(b, num_token, self.num_heads, self.head_dim)
    values = values.view(b, num_token, self.num_heads, self.head_dim)
    queries = queries.view(b, num_token, self.num_heads, self.head_dim)

    # (b, num_token, num_head, head_dim) -> (b, num_head, num_token, head_dim) to get a better hand on doing multiplications
    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    queries = queries.transpose(1,2)

    # The main multiplication happens between (..., num_token, head_dim) and (..., head_dim, num_tokens)
    attn_scores = queries @ keys.transpose(2,3) # we get (..., num_token, num_token)

    # MASKINGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG and getting attention weights
    masked_bool = self.mask.bool()[:num_token, :num_token]
    attn_scores.masked_fill(masked_bool, -torch.inf)
    attn_scores = attn_scores / keys.shape[-1]**0.5

    attn_weights = torch.softmax(attn_scores, dim=-1)
    attn_weights = self.dropout(attn_weights)

    # After getting context_vec, we do (b, num_head, num_token, head_dim) -> (b, num_token, num_head, head_dim)
    context_vec = (attn_weights @ values).transpose(1,2)

    # And finally we eat last two dimensions to (b, num_token, num_head, head_dim) -> (b, num_token, d_out)
    context_vec = context_vec.contiguous().view(b, num_token, self.d_out)

    return context_vec

In [8]:
ma = MultiHeadAttention(3, 2, 6, 0.0, 2)

In [9]:
ma(batch)

tensor([[[-0.0137,  0.5634],
         [-0.0137,  0.5643],
         [-0.0137,  0.5643],
         [-0.0136,  0.5647],
         [-0.0137,  0.5645],
         [-0.0136,  0.5647]],

        [[-0.0137,  0.5634],
         [-0.0137,  0.5643],
         [-0.0137,  0.5643],
         [-0.0136,  0.5647],
         [-0.0137,  0.5645],
         [-0.0136,  0.5647]]], grad_fn=<ViewBackward0>)